## in this section we will start with the IMDB data and make the most common processing algorithm Word2Vec with Gensim packge, what is word2vec we already talk about it in previous chapters but in this chapter we try to use it with IMDB dataset, so lets take a quick tour in this section we first have some data which is negative and positive and we will dived them into train and test but the new thing that we will do, is to make it as a numbers as we talk in Word Embedding, as we talk about it in previous section then after change them, we will pass it to a new layer to feed the learning layer to complete the learning task or save it in a pickle, as we do in this case we try to allow you learn how to use this genesim embedding, and have the concept of word2vec, so you can then learn more about  Doc2vec, also this a very used cases in buliding good models. 

## Import packages

In [0]:
!unzip /content/aclImdb.zip

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import html
import os
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import re
from tqdm import tqdm
#print(os.listdir("../input"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


we need now to load files positive ones, negative ones, and test files 

In [0]:
path = "/content/aclImdb/"
positiveFiles = [x for x in os.listdir(path+"train/pos/")
                 if x.endswith(".txt")]
negativeFiles = [x for x in os.listdir(path+"train/neg/")
                if x.endswith(".txt")]
testFiles = [x for x in os.listdir(path+"test/") 
             if x.endswith(".txt")]

positiveFiles as it is a positive reviews 

In [0]:
positiveReviews, negativeReviews, testReviews = [], [], []
for pfile in positiveFiles:
    with open(path+"train/pos/"+pfile, encoding="latin1") as f:
        positiveReviews.append(f.read())
for nfile in negativeFiles:
    with open(path+"train/neg/"+nfile, encoding="latin1") as f:
        negativeReviews.append(f.read())
for tfile in testFiles:
    with open(path+"test/"+tfile, encoding="latin1") as f:
        testReviews.append(f.read())

the size of every review type, we now need to know the size of positive reviews and negative reviews

In [0]:
print(len(positiveReviews))
print(len(negativeReviews))
print(len(testReviews))

12500
12500
2


let's make all types of reviews in same dataframe so we can see it.

In [0]:
reviews = pd.concat([pd.DataFrame({"review":positiveReviews, "label":1,
                                   "file":positiveFiles}),
                    pd.DataFrame({"review":negativeReviews, "label":0,
                                   "file":negativeFiles}),
                    pd.DataFrame({"review":testReviews, "label":-1,
                                   "file":testFiles})
                    ], ignore_index=True).sample(frac=1, random_state=1)

get the data shape is should be number with three as the dimension of data 

In [0]:
reviews.shape

(25002, 3)

lets see the dataframe of data 

In [0]:
reviews[0:10]

,review,label,file
21939,"Oh my god, what a horrible film. The film has ...",0,3942_1.txt
24113,There should be a rule that states quite clear...,0,7657_1.txt
4633,I found this to be a so-so romance/drama that ...,1,16_7.txt
17240,Forest of the Damned starts out as five young ...,0,10516_4.txt
4894,They had me from the first show.<br /><br />We...,1,5665_9.txt
6908,saw this in preview- great movie- wonderful ch...,1,4333_10.txt
19498,First of all this movie starts out on a really...,0,5486_3.txt
8146,I've been a fan of Xu Ke (Hark Tsui) for many ...,1,405_10.txt
13815,I can't say much about this film. I think it s...,0,1834_3.txt
6483,<br /><br />Everything is relative seems to be...,1,8796_8.txt


stop words in english to be ignored

In [0]:
stopWords = stopwords.words('english')

define function that make some cleaning process

In [0]:
def CleanData(sentence):
    processedList = ""
    
    #convert to lowercase and ignore special charcter
    sentence = re.sub(r'[^A-Za-z0-9\s.]', r'', str(sentence).lower())
    sentence = re.sub(r'\n', r' ', sentence)
    
    sentence = " ".join([word for word in sentence.split() if word not in stopWords])
    
    return sentence

In [0]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25002 entries, 21939 to 235
Data columns (total 3 columns):
review    25002 non-null object
label     25002 non-null int64
file      25002 non-null object
dtypes: int64(1), object(2)
memory usage: 781.3+ KB


In [0]:
reviews['review'][0]

'Level One, Horror.<br /><br />When I saw this film for the first time at 10, I knew it would give me nightmares. It did. Surprisingly, as I recall, it was the sound as much as the sight of the monster that caused them.<br /><br />Level Two, Psychoanalytic Theory.<br /><br />Later as an adult, I saw the story for what it was: What if the savage, unrestrained instincts we all repress became manifest.<br /><br />Level Three, Pure Science Fiction.<br /><br />The best way plausibly to realize the plot\'s "What if" is through the science fiction genre. This is pure science fiction, not the "cowboys in space" that passes for the genre today.<br /><br />After 43 years, Forbidden Planet remains the greatest of all science fiction films. If planning a remake, SKG or Lucas, Watch Out!'

In [0]:
CleanData(reviews['review'][0])

'level one horror.br br saw film first time 10 knew would give nightmares. did. surprisingly recall sound much sight monster caused them.br br level two psychoanalytic theory.br br later adult saw story savage unrestrained instincts repress became manifest.br br level three pure science fiction.br br best way plausibly realize plots science fiction genre. pure science fiction cowboys space passes genre today.br br 43 years forbidden planet remains greatest science fiction films. planning remake skg lucas watch'

In [0]:
reviews['review'] = reviews['review'].map(lambda x: CleanData(x))

In [0]:
reviews['review'].head()

21939    oh god horrible film. film right people involv...
24113    rule states quite clearly movies like resident...
4633     found soso romancedrama nice ending generally ...
17240    forest damned starts five young friends brothe...
4894     first show.br br welcome trinity county. sleep...
Name: review, dtype: object

In [0]:
tmp_corpus = reviews['review'].map(lambda x:x.split('.'))

In [0]:
#corpus [[w1, w2, w3,...],[...]]
corpus = []

for i in tqdm(range(len(reviews))):
    for line in tmp_corpus[i]:
        words = [x for x in line.split()]
        corpus.append(words)

100%|██████████| 25002/25002 [00:02<00:00, 10673.63it/s]


In [0]:
len(corpus)

402194

In [0]:
#removing blank list
corpus_new = []
for i in range(len(corpus)):
    if (len(corpus[i]) != 0):
        corpus_new.append(corpus[i])

In [0]:
num_of_sentences = len(corpus_new)
num_of_words = 0
for line in corpus_new:
    num_of_words += len(line)

print('Num of sentences - %s'%(num_of_sentences))
print('Num of words - %s'%(num_of_words))

Num of sentences - 354417
Num of words - 3265546


now let's see the gensim package and how to use it in the word2vec

In [0]:
from gensim.models import Word2Vec

let's bulid a word2vec model and insialize parameters

In [0]:
# sg - skip gram |  window = size of the window | size = vector dimension
size = 100
window_size = 2 # sentences weren't too long, so
epochs = 100
min_count = 2
workers = 4

model = Word2Vec(corpus_new)

In [0]:
model.build_vocab(sentences= corpus_new, update=True)

for i in range(5):
    model.train(sentences=corpus_new, epochs=50, total_examples=model.corpus_count)

after model trained let's save it

In [0]:
#save model
model.save('w2v_model')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


load model into WOrd2Vec which is a modle in gensim

In [0]:
model = Word2Vec.load('w2v_model')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


lets get the most similar movies 

In [0]:
model.wv.most_similar('movie')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('film', 0.8756906986236572),
 ('flick', 0.6631126403808594),
 ('movies', 0.6589803695678711),
 ('it', 0.562816321849823),
 ('films', 0.5470719337463379),
 ('show', 0.5167748928070068),
 ('sequel', 0.5143758654594421),
 ('this', 0.5129573941230774),
 ('thing', 0.5066217184066772),
 ('really', 0.4848993122577667)]

the next step we will extract data using its label

In [0]:
reviews = reviews[["review", "label", "file"]].sample(frac=1,
                                                      random_state=1)
train = reviews[reviews.label!=-1].sample(frac=0.6, random_state=1)
valid = reviews[reviews.label!=-1].drop(train.index)
test = reviews[reviews.label==-1]

shapes of every type of data

In [0]:
print(train.shape)
print(valid.shape)
print(test.shape)

(15000, 3)
(10000, 3)
(2, 3)


In [0]:
valid.head()

,review,label,file
21572,beating bad guys... tag line movie exposes muc...,0,12438_1.txt
1806,title sequence shows credits written rainsoake...,1,6741_7.txt
4753,diego armando maradona still remains best foot...,1,4968_10.txt
18029,even first 10 minutes movie horrific. hard bel...,0,2137_1.txt
20189,hazing confused mumbojumbo wants hard evil dea...,0,11400_4.txt


some data preprocessing which the last iteration before we train our model

In [0]:
num_features = 100

In [0]:
index2word_set = set(model.wv.index2word)

In [0]:
model = model

In [0]:
def featureVecorMethod(words):
    featureVec = np.zeros(num_features, dtype='float32')
    nwords = 0
    
    for word in words:
        if word in index2word_set:
            nwords+= 1
            featureVec = np.add(featureVec, model[word])
            
    #average of feature vec
    featureVec = np.divide(featureVec, nwords)
    return featureVec

In [0]:
def getAvgFeatureVecs(reviews):
    counter = 0
    
    reviewFeatureVecs = np.zeros((len(reviews), num_features), dtype='float32')
    for review in reviews:
#         print(review)
        if counter%1000 == 0:
            print("Review %d of %d"%(counter, len(reviews)))
            
        reviewFeatureVecs[counter] = featureVecorMethod(review)
        counter = counter+1
    return reviewFeatureVecs

In [0]:
clean_train_reviews = []
for review in train['review']:
#     print(review)
    clean_train_reviews.append(list(CleanData(review).split()))
# print(len(clean_train_reviews))\

trainDataVecs = getAvgFeatureVecs(clean_train_reviews)

Review 0 of 15000


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Review 1000 of 15000
Review 2000 of 15000
Review 3000 of 15000
Review 4000 of 15000
Review 5000 of 15000
Review 6000 of 15000
Review 7000 of 15000
Review 8000 of 15000
Review 9000 of 15000
Review 10000 of 15000
Review 11000 of 15000
Review 12000 of 15000
Review 13000 of 15000
Review 14000 of 15000


In [0]:
len(valid['review'])

10000

In [0]:
clean_test_reviews = []
for review in valid['review']:
#     print(review)
    clean_test_reviews.append(list(CleanData(review).split()))
# print(len(clean_train_reviews))\

testDataVecs = getAvgFeatureVecs(clean_test_reviews)

Review 0 of 10000


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Review 1000 of 10000
Review 2000 of 10000
Review 3000 of 10000
Review 4000 of 10000
Review 5000 of 10000
Review 6000 of 10000
Review 7000 of 10000
Review 8000 of 10000
Review 9000 of 10000


In [0]:
print(len(testDataVecs))

10000
